# rf502_wspacewrite
Organisation and simultaneous fits: creating and writing a workspace




**Author:** Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Saturday, November 16, 2024 at 10:12 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooChebychev.h"
#include "RooAddPdf.h"
#include "RooWorkspace.h"
#include "RooPlot.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "TFile.h"
#include "TH1.h"
using namespace RooFit;

Create model and dataset
-----------------------------------------------

Declare observable x

In [2]:
RooRealVar x("x", "x", 0, 10);

Create two Gaussian PDFs g1(x,mean1,sigma) anf g2(x,mean2,sigma) and their parameters

In [3]:
RooRealVar mean("mean", "mean of gaussians", 5, 0, 10);
RooRealVar sigma1("sigma1", "width of gaussians", 0.5);
RooRealVar sigma2("sigma2", "width of gaussians", 1);

RooGaussian sig1("sig1", "Signal component 1", x, mean, sigma1);
RooGaussian sig2("sig2", "Signal component 2", x, mean, sigma2);

[#0] WARNING:InputArguments -- The parameter 'sigma1' with range [-inf, inf] of the RooGaussian 'sig1' exceeds the safe range of (0, inf). Advise to limit its range.
[#0] WARNING:InputArguments -- The parameter 'sigma2' with range [-inf, inf] of the RooGaussian 'sig2' exceeds the safe range of (0, inf). Advise to limit its range.


Build Chebychev polynomial pdf

In [4]:
RooRealVar a0("a0", "a0", 0.5, 0., 1.);
RooRealVar a1("a1", "a1", 0.2, 0, 1.);
RooChebychev bkg("bkg", "Background", x, RooArgSet(a0, a1));

Sum the signal components into a composite signal pdf

In [5]:
RooRealVar sig1frac("sig1frac", "fraction of component 1 in signal", 0.8, 0., 1.);
RooAddPdf sig("sig", "Signal", RooArgList(sig1, sig2), sig1frac);

Sum the composite signal and background

In [6]:
RooRealVar bkgfrac("bkgfrac", "fraction of background", 0.5, 0., 1.);
RooAddPdf model("model", "g1+g2+a", RooArgList(bkg, sig), bkgfrac);

Generate a data sample of 1000 events in x from model

In [7]:
std::unique_ptr<RooDataSet> data{model.generate(x, 1000)};

input_line_55:2:2: warning: 'data' shadows a declaration with the same name in the 'std' namespace; use '::data' to reference this declaration
 std::unique_ptr<RooDataSet> data{model.generate(x, 1000)};
 ^


Create workspace, import data and model
-----------------------------------------------------------------------------

Create a new empty workspace

In [8]:
RooWorkspace *w = new RooWorkspace("w", "workspace");

Import model and all its components into the workspace

In [9]:
w->import(model);

[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooAddPdf::model
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooChebychev::bkg
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::x
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::a0
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::a1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::bkgfrac
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooAddPdf::sig
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooGaussian::sig1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::mean
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::sigma1
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooRealVar::sig1frac
[#1] INFO:ObjectHandling -- RooWorkspace::import(w) importing RooGaussian::sig2
[#1] INFO:ObjectHandling -- RooWorkspace::import(

Import data into the workspace

In [10]:
w->import(*data);

input_line_58:2:13: error: reference to 'data' is ambiguous
 w->import(*data);
            ^
input_line_55:2:30: note: candidate found by name lookup is 'data'
 std::unique_ptr<RooDataSet> data{model.generate(x, 1000)};
                             ^
/usr/include/c++/9/bits/range_access.h:318:5: note: candidate found by name lookup is 'std::data'
    data(initializer_list<_Tp> __il) noexcept
    ^
/usr/include/c++/9/bits/range_access.h:289:5: note: candidate found by name lookup is 'std::data'
    data(_Container& __cont) noexcept(noexcept(__cont.data()))
    ^
/usr/include/c++/9/bits/range_access.h:299:5: note: candidate found by name lookup is 'std::data'
    data(const _Container& __cont) noexcept(noexcept(__cont.data()))
    ^
/usr/include/c++/9/bits/range_access.h:309:5: note: candidate found by name lookup is 'std::data'
    data(_Tp (&__array)[_Nm]) noexcept
    ^


Print workspace contents

In [11]:
w->Print();


RooWorkspace(w) workspace contents

variables
---------
(a0,a1,bkgfrac,mean,sig1frac,sigma1,sigma2,x)

p.d.f.s
-------
RooChebychev::bkg[ x=x coefList=(a0,a1) ] = 0.8
RooAddPdf::model[ bkgfrac * bkg + [%] * sig ] = 0.9/1
RooAddPdf::sig[ sig1frac * sig1 + [%] * sig2 ] = 1/1
RooGaussian::sig1[ x=x mean=mean sigma=sigma1 ] = 1
RooGaussian::sig2[ x=x mean=mean sigma=sigma2 ] = 1



Save workspace in file
-------------------------------------------

Save the workspace into a ROOT file

In [12]:
w->writeToFile("rf502_workspace.root");

Workspace will remain in memory after macro finishes

In [13]:
gDirectory->Add(w);